### Direct Search

In [ ]:
# Libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# EMA
from ema_workbench import (Model, RealParameter,  MultiprocessingEvaluator, TimeSeriesOutcome, perform_experiments, ema_logging)
from ema_workbench import Samplers
from ema_workbench import save_results, load_results
from ema_workbench.analysis import feature_scoring
from ema_workbench.analysis.scenario_discovery_util import RuleInductionType
from ema_workbench.em_framework.optimization import EpsilonProgress, HyperVolume
from ema_workbench.em_framework.salib_samplers import get_SALib_problem

# Analytical Tools
from SALib.analyze import sobol

# Model
from problem_formulation import get_model_for_problem_formulation
ema_logging.log_to_stderr(ema_logging.INFO)

### Initialize Model

In [ ]:
# Set Model Parameters
model, steps = get_model_for_problem_formulation(1)

uncertainties = model.uncertainties
levers = model.levers
outcomes = model.outcomes

# Reference Scenario
ref_val = {'Bmax': 175, 
           'Brate': 1.5, 
           'pfail': 0.5,
            'discount rate': 3.5,
            'ID flood wave shape': 4}

ref_dict = {}
# < ref_dict >
# reference scenario updated for all dike rings
for key in model.uncertainties:
    name_split = key.name.split('_')
    if len(name_split) == 1:
        if key.name in ref_val.keys():
            ref_dict.update({key.name: ref_val[name]})
    else:
        ref_dict.update({key.name: ref_val[name_split[1]]})

ref_scenario = Scenario('reference', **ref_dict)

# Optimization Parameters

convergence_metrics = [EpsilonProgress()]
nfe = 100000

In [ ]:
ema_logging.log_to_stderr(ema_logging.INFO)
with MultiprocessingEvaluator(model) as evaluator:
         results, convergence = evaluator.optimize(nfe=nfe,
                                                  searchover='levers',
                                                   epsilons=[0.1,]*len(model.outcomes),
                                                   convergence=convergence_metrics,
                                                   reference=ref_scenario
                                                   )

### Run Optimization on Lever Space
Find candidate policies

In [ ]:
results=

### Reduction of pareto set of candiate policies at hand criteria-extent distribution

### Robustness analysis


### Visualization for Robustness